In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv
/kaggle/input/playground-series-s5e6/sample_submission.csv
/kaggle/input/playground-series-s5e6/train.csv
/kaggle/input/playground-series-s5e6/test.csv


In [2]:
!pip install -q scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 94.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [3]:
from lightgbm import LGBMClassifier, log_evaluation, early_stopping
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import joblib
import optuna
import shutil
import glob
import json
import gc

warnings.filterwarnings("ignore")

In [4]:
class CFG:
    train_path = "/kaggle/input/playground-series-s5e6/train.csv"
    test_path = "/kaggle/input/playground-series-s5e6/test.csv"
    sample_sub_path = "/kaggle/input/playground-series-s5e6/sample_submission.csv"

    original_path = "/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv"
    
    target = "Fertilizer Name"
    n_folds = 5
    seed = 42

Data loading and preprocessing

In [5]:
train = pd.read_csv(CFG.train_path, index_col="id")
test = pd.read_csv(CFG.test_path, index_col="id")
original = pd.read_csv(CFG.original_path)

features = test.columns.tolist()
train[features] = train[features].astype(str).astype("category")
test[features] = test[features].astype(str).astype("category")
original[features] = original[features].astype(str).astype("category")

label_encoder = LabelEncoder()
train[CFG.target] = label_encoder.fit_transform(train[CFG.target])
original[CFG.target] = label_encoder.transform(original[CFG.target])

X = train.drop(CFG.target, axis=1)
y = train[CFG.target]
X_test = test

X_original = original.drop(CFG.target, axis=1)
y_original = original[CFG.target]

In [6]:
mutual_info = mutual_info_classif(X, y, random_state=CFG.seed, discrete_features=True)

mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns
mutual_info = pd.DataFrame(mutual_info.sort_values(ascending=False), columns=['Mutual Information'])
mutual_info.style.bar(subset=['Mutual Information'], cmap='RdYlGn')

,Mutual Information
Moisture,0.003308
Phosphorous,0.002984
Nitrogen,0.002373
Crop Type,0.002088
Potassium,0.001367
Humidity,0.001015
Temparature,0.000966
Soil Type,0.000617


In [7]:
mutual_info = mutual_info_classif(X_original, y_original, random_state=CFG.seed, discrete_features=True)

mutual_info = pd.Series(mutual_info)
mutual_info.index = X_original.columns
mutual_info = pd.DataFrame(mutual_info.sort_values(ascending=False), columns=['Mutual Information'])
mutual_info.style.bar(subset=['Mutual Information'], cmap='RdYlGn')

,Mutual Information
Phosphorous,0.001362
Moisture,0.001355
Nitrogen,0.001068
Potassium,0.000626
Humidity,0.000494
Temparature,0.000432
Crop Type,0.000320
Soil Type,0.000107


Training base models

In [8]:
def map3(y_true, y_pred_probs):
    y_true = [[x] for x in y_true]
    y_pred_probs = np.argsort(y_pred_probs, axis=1)[:, -3:][:, ::-1].tolist()
    
    def ap3(y_true, y_pred_probs):
        y_pred_probs = y_pred_probs[:3]

        score = 0.0
        num_hits = 0.0

        for i,p in enumerate(y_pred_probs):
            if p in y_true and p not in y_pred_probs[:i]:
                num_hits += 1.0
                score += num_hits / (i+1.0)

        if not y_true:
            return 0.0

        return score
    
    return np.mean([ap3(a,p) for a,p in zip(y_true, y_pred_probs)])

In [9]:
class Trainer:
    def __init__(self, model, config=CFG):
        self.model = model
        self.config = config

    def fit_predict(self, X, y, X_test, X_original=None, y_original=None, fit_args={}):
        print(f"Training {self.model.__class__.__name__}\n")
        
        scores = []        
        oof_pred_probs = np.zeros((X.shape[0], 7))
        test_pred_probs = np.zeros((X_test.shape[0], 7))
        
        skf = StratifiedKFold(n_splits=self.config.n_folds, random_state=self.config.seed, shuffle=True)
        for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            if X_original is not None and y_original is not None:
                X_train = pd.concat([X_train, X_original])
                y_train = pd.concat([y_train, y_original])
            
            model = clone(self.model)
            
            if fit_args:
                model.fit(X_train, y_train, **fit_args, eval_set=[(X_val, y_val)])
            else:
                model.fit(X_train, y_train)
            
            y_pred_probs = model.predict_proba(X_val)
            oof_pred_probs[val_idx] = y_pred_probs
            
            temp_test_pred_probs = model.predict_proba(X_test)
            test_pred_probs += temp_test_pred_probs / self.config.n_folds
            
            score = map3(y_val, y_pred_probs)
            scores.append(score)
            
            del model, X_train, y_train, X_val, y_val, y_pred_probs
            gc.collect()
            
            if fit_args:
                print(f"\n--- Fold {fold_idx + 1} - MAP@3: {score:.6f}\n\n")
            else:
                print(f"--- Fold {fold_idx + 1} - MAP@3: {score:.6f}")
                            
        overall_score = map3(y, oof_pred_probs)
            
        print(f"\n------ Overall MAP@3: {overall_score:.6f} | Average MAP@3: {np.mean(scores):.6f} ± {np.std(scores):.6f}")
        
        return oof_pred_probs, test_pred_probs, scores

    def tune(self, X, y):        
        scores = []        
        
        skf = StratifiedKFold(n_splits=self.config.n_folds, random_state=self.config.seed, shuffle=True)
        for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            model = clone(self.model)
            model.fit(X_train, y_train)
            
            y_pred_probs = model.predict_proba(X_val)            
            score = map3(y_val, y_pred_probs)
            scores.append(score)
            
            del model, X_train, y_train, X_val, y_val, y_pred_probs
            gc.collect()
            
        return np.mean(scores)

In [10]:
lgbm_params = {
    "boosting_type": "gbdt",
    "device": "gpu",
    "colsample_bytree": 0.4366677273946288,
    "learning_rate": 0.016164161953515117,
    "max_depth": 12,
    "min_child_samples": 67,
    "n_estimators": 10000,
    "n_jobs": -1,
    "num_leaves": 243,
    "random_state": 42,
    "reg_alpha": 6.38288560443373,
    "reg_lambda": 9.392999314379155,
    "subsample": 0.7989164499431718,
    "verbose": -1
}

lgbm_goss_params = {
    "boosting_type": "goss",
    "device": "gpu",
    "colsample_bytree": 0.32751831793031183,
    "learning_rate": 0.006700715059604966,
    "max_depth": 12,
    "min_child_samples": 84,
    "n_estimators": 10000,
    "n_jobs": -1,
    "num_leaves": 229,
    "random_state": 42,
    "reg_alpha": 6.879977008084246,
    "reg_lambda": 4.739518466581721,
    "subsample": 0.5411572049978781,
    "verbose": -1
}

xgb_params = {
    # https://www.kaggle.com/code/patrykszcz/simple-xgb-model-5-fold
    "device": "gpu",
    "max_depth": 12,
    "colsample_bytree": 0.467,
    "subsample": 0.86,
    "n_estimators": 10000,
    "learning_rate": 0.03,
    "gamma": 0.26,
    "max_delta_step": 4,
    "reg_alpha": 2.7,
    "reg_lambda": 1.4,
    "early_stopping_rounds": 100,
    "objective": 'multi:softprob',
    "random_state": 13,
    "enable_categorical": True,
}

In [11]:
scores = {}
oof_pred_probs = {}
test_pred_probs = {}

LightGBM (gbdt)

In [12]:
lgbm_model = LGBMClassifier(**lgbm_params)
lgbm_trainer = Trainer(lgbm_model)

fit_args = {
    "callbacks": [
        log_evaluation(period=1000), 
        early_stopping(stopping_rounds=100)
    ]
}

oof_pred_probs["LightGBM (gbdt)"], test_pred_probs["LightGBM (gbdt)"], scores["LightGBM (gbdt)"] = lgbm_trainer.fit_predict(X, y, X_test, X_original, y_original, fit_args)

Training LGBMClassifier



1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 100 rounds
[1000]	valid_0's multi_logloss: 1.89544
Early stopping, best iteration is:
[1799]	valid_0's multi_logloss: 1.89043

--- Fold 1 - MAP@3: 0.365319


Training until validation scores don't improve for 100 rounds
[1000]	valid_0's multi_logloss: 1.89535
[2000]	valid_0's multi_logloss: 1.89001
Early stopping, best iteration is:
[1909]	valid_0's multi_logloss: 1.88997

--- Fold 2 - MAP@3: 0.365950


Training until validation scores don't improve for 100 rounds
[1000]	valid_0's multi_logloss: 1.89375
[2000]	valid_0's multi_logloss: 1.88771
Early stopping, best iteration is:
[2082]	valid_0's multi_logloss: 1.88767

--- Fold 3 - MAP@3: 0.368083


Training until validation scores don't improve for 100 rounds
[1000]	valid_0's multi_logloss: 1.89599
Early stopping, best iteration is:
[1898]	valid_0's multi_logloss: 1.89066

--- Fold 4 - MAP@3: 0.365314


Training until validation scores don't improve for 100 rounds
[1000]	valid_0's multi

LightGBM (goss)

In [13]:
lgb_goss_model = LGBMClassifier(**lgbm_goss_params)
lgb_goss_trainer = Trainer(lgb_goss_model)

fit_args = {
    "callbacks": [
        log_evaluation(period=1000), 
        early_stopping(stopping_rounds=100)
    ]
}

oof_pred_probs["LightGBM (goss)"], test_pred_probs["LightGBM (goss)"], scores["LightGBM (goss)"] = lgb_goss_trainer.fit_predict(X, y, X_test, X_original, y_original, fit_args)

Training LGBMClassifier

Training until validation scores don't improve for 100 rounds
[1000]	valid_0's multi_logloss: 1.90751
[2000]	valid_0's multi_logloss: 1.8965
[3000]	valid_0's multi_logloss: 1.8922
Early stopping, best iteration is:
[3700]	valid_0's multi_logloss: 1.89154

--- Fold 1 - MAP@3: 0.364424


Training until validation scores don't improve for 100 rounds
[1000]	valid_0's multi_logloss: 1.90778
[2000]	valid_0's multi_logloss: 1.89652
[3000]	valid_0's multi_logloss: 1.892
Early stopping, best iteration is:
[3490]	valid_0's multi_logloss: 1.89114

--- Fold 2 - MAP@3: 0.364682


Training until validation scores don't improve for 100 rounds
[1000]	valid_0's multi_logloss: 1.90673
[2000]	valid_0's multi_logloss: 1.89513
[3000]	valid_0's multi_logloss: 1.89031
Early stopping, best iteration is:
[3696]	valid_0's multi_logloss: 1.88911

--- Fold 3 - MAP@3: 0.366162


Training until validation scores don't improve for 100 rounds
[1000]	valid_0's multi_logloss: 1.90796
[2000]	val

XGBoost

In [14]:
xgb_model = XGBClassifier(**xgb_params)
xgb_trainer = Trainer(xgb_model)

fit_args = {
    "verbose": 1000
}

oof_pred_probs["XGBoost"], test_pred_probs["XGBoost"], scores["XGBoost"] = xgb_trainer.fit_predict(X, y, X_test, X_original, y_original, fit_args)

Training XGBClassifier

[0]	validation_0-mlogloss:1.94553
[912]	validation_0-mlogloss:1.88990

--- Fold 1 - MAP@3: 0.366436


[0]	validation_0-mlogloss:1.94553
[895]	validation_0-mlogloss:1.88909

--- Fold 2 - MAP@3: 0.366754


[0]	validation_0-mlogloss:1.94553
[946]	validation_0-mlogloss:1.88769

--- Fold 3 - MAP@3: 0.368023


[0]	validation_0-mlogloss:1.94553
[871]	validation_0-mlogloss:1.89012

--- Fold 4 - MAP@3: 0.365927


[0]	validation_0-mlogloss:1.94552
[883]	validation_0-mlogloss:1.88864

--- Fold 5 - MAP@3: 0.366706



------ Overall MAP@3: 0.366769 | Average MAP@3: 0.366769 ± 0.000693


LogisticRegression

In [15]:

from itertools import combinations
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mutual_info_score
from sklearn.utils.parallel import Parallel, delayed
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin, clone

X = pd.read_csv(CFG.train_path, index_col='id')
X_test = pd.read_csv(CFG.test_path, index_col='id')

le = LabelEncoder()
y = X.pop('Fertilizer Name')
y = le.fit_transform(y)

X = X.astype(str)
X_test = X_test.astype(str)

def adjusted_mutual_info(x, y, n_iter=5):
    x, y = x.astype(str), y.astype(str)
    m0 = mutual_info_score(x, y)
    m1 = Parallel(n_jobs=-1)(
        delayed(lambda rs: mutual_info_score(
            y, np.random.default_rng(rs).permutation(x)
        ))(rs)
        for rs in range(n_iter)
    )
    return m0 - np.mean(m1)

mi = {}
e =  mutual_info_score(y, y)
for c1, c2, c3 in tqdm(list(combinations(list(X.columns), 3))):
    c = c1+'_'+c2+'_'+c3
    mi[c] = adjusted_mutual_info(X[c1]+'_'+X[c2]+'_'+X[c3], y)/e

comb3 = sorted(mi, key=mi.get, reverse=True)

def Augmented(model, X_o, y_o, weight_arg='sample_weight', weight=1.0):
    class AugmentedModel(ClassifierMixin, BaseEstimator):
        def fit(self, X, y):
            sample_weight = np.array([1.0]*len(X)+[weight]*len(X_o))
            X = pd.concat([X, X_o])
            y = np.concatenate([y, y_o])
            self.m = clone(model).fit(X, y, **{weight_arg: sample_weight})
            self.classes_ = self.m.classes_
            return self
        def predict_proba(self, X):
            return self.m.predict_proba(X)
    return AugmentedModel()

X_o = pd.read_csv(CFG.original_path)
y_o = le.transform(X_o.pop('Fertilizer Name'))

X_all = pd.concat([X, X_o]).astype(str)

X_all_e = X_all.copy()
X_test_e = X_test.copy()
for c1, c2 in combinations(X_all.columns, 2):
    X_all_e[c1+'_'+c2] = X_all[c1]+'_'+X_all[c2]
    X_test_e[c1+'_'+c2] = X_test[c1]+'_'+X_test[c2]

topk = 25
for c1_c2_c3 in comb3[:topk]:
    c1, c2, c3 = c1_c2_c3.split('_')
    X_all_e[c1_c2_c3] = X_all[c1]+'_'+X_all[c2]+'_'+X_all[c3]
    X_test_e[c1_c2_c3] = X_test[c1]+'_'+X_test[c2]+'_'+X_test[c3]

X_e = X_all_e.iloc[:len(X)]
X_o_e = X_all_e.iloc[len(X):]

100%|██████████| 56/56 [11:34<00:00, 12.41s/it]


In [16]:
lr_model = Augmented(
    make_pipeline(
        OneHotEncoder(handle_unknown='ignore'),
        LogisticRegression(C=1e-2, max_iter=10000, random_state=0)
    ), X_o_e, y_o, weight_arg='logisticregression__sample_weight', 
    weight=4.0
)

lr_trainer = Trainer(lr_model)
oof_pred_probs["LogisticRegression"], test_pred_probs["LogisticRegression"], scores["LogisticRegression"] = lr_trainer.fit_predict(X_e, y, X_test_e)

Training AugmentedModel

--- Fold 1 - MAP@3: 0.374382
--- Fold 2 - MAP@3: 0.373953
--- Fold 3 - MAP@3: 0.376254
--- Fold 4 - MAP@3: 0.373357
--- Fold 5 - MAP@3: 0.375070

------ Overall MAP@3: 0.374603 | Average MAP@3: 0.374603 ± 0.000997


Ensembling with LogisticRegression

In [17]:
X = pd.DataFrame(np.concatenate(list(oof_pred_probs.values()), axis=1))
X_test = pd.DataFrame(np.concatenate(list(test_pred_probs.values()), axis=1))

In [18]:
joblib.dump(X, "oof_pred_probs.pkl")
joblib.dump(X_test, "test_pred_probs.pkl")

['test_pred_probs.pkl']

In [19]:
lr_model = LogisticRegression(
    random_state=42, 
    max_iter=1000, 
    solver='liblinear', 
    penalty='l2', 
    C=32.89802104596641,
    tol=0.0029878837974181643,
    fit_intercept=True
) 
lr_trainer = Trainer(lr_model)
_, lr_test_pred_probs, scores["Ensemble"] = lr_trainer.fit_predict(X, y, X_test)

Training LogisticRegression

--- Fold 1 - MAP@3: 0.375827
--- Fold 2 - MAP@3: 0.375341
--- Fold 3 - MAP@3: 0.377570
--- Fold 4 - MAP@3: 0.374616
--- Fold 5 - MAP@3: 0.375921

------ Overall MAP@3: 0.375855 | Average MAP@3: 0.375855 ± 0.000974


In [20]:
final_predictions = []

In [21]:
final_predictions = []
for i in np.argsort(lr_test_pred_probs)[:, -3:][:, ::-1]:
    prediction = label_encoder.inverse_transform(i)
    final_predictions.append(" ".join(prediction))

In [22]:
sub = pd.read_csv(CFG.sample_sub_path)
sub[CFG.target] = final_predictions
sub.to_csv(f"sub_logistic-regression_{np.mean(scores['Ensemble']):.6f}.csv", index=False)
sub.head()

,id,Fertilizer Name
0,750000,10-26-26 20-20 DAP
1,750001,17-17-17 28-28 10-26-26
2,750002,20-20 Urea DAP
3,750003,14-35-14 17-17-17 10-26-26
4,750004,Urea 20-20 28-28
